In [ ]:
from nbdev import *
%nbdev_default_export functions

Cells will be exported to pct.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Functions

Functions that form the elements of a perceptual control node (system).

Each outputs the result of applying the function and may be a scalar or array, depending on the input. The inputs are supplied as links to another function, from where the values are read. Unless the function is a simple case, such as a 'Constant'. 

Parameters are supplied as arguments to the constructor of the class.

In [ ]:
%nbdev_hide
import numpy as np

In [ ]:
%nbdev_export
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function. This class is not used direclty by developers, but defines the functionality common to all."
    def __init__(self, name, value):
        self.value = value
        self.links = []
        self.name = name
        
    @abstractmethod
    def __call__(self, verbose=False):
        if verbose :
            print(f'{self.value}', end= " ")
            
        return self.value
    
    @abstractmethod    
    def summary(self, str):
        print(f'{self.name} {type(self).__name__}', end = " ")
        if len(str)>0:
            print(f'| {str}', end= " ")
        print(f'| {self.value}', end = " ")
        if len(self.links)>0:
            print(f'| links ', end=" ")
        for link in self.links:
            print(link.get_name(), end= " ")
        print()
        
    @abstractmethod    
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name, 
                    "value": self.value}
        return config
        
    def get_name(self):
        return self.name
    
    def set_name(self, name):
        self.name=name

    def set_value(self, value):
        self.value= value
    
    def get_value(self):
        return self.value
    
    def add_link(self, linkfn):
        self.links.append(linkfn)
    
    #def from_config(cls, config):
    @classmethod
    def from_config(cls, config):
        #cls = config.pop("type")
        return cls(**config)


In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "A proportion of the input value as defined by the gain parameter."
    def __init__(self, gain=1, name="proportional", value=0, **cargs):
        super(Proportional, self).__init__(name, value)
        self.gain = gain

    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = input * self.gain
        return super().__call__(verbose)
    
    def summary(self):
        super().summary(f'gain {self.gain}')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        return config               

# Creating Functions

Standard class constructor. Different ways to create a function with the standard constructor.

In [ ]:
prop = Proportional()
print(prop.get_config())
prop = Proportional(10, name="myprop")
print(prop.get_config())
prop = Proportional(gain=10)
print(prop.get_config())

{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'gain': 1}
{'type': 'Proportional', 'name': 'myprop', 'value': 0, 'gain': 10}
{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'gain': 10}


Configuration class constructor. Create the function by passing a configuration structure to the constructor.

In [ ]:
prop = Proportional(**{'name': 'myprop', 'value': 5, 'gain': 20})
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop', 'value': 5, 'gain': 20}


Configuration class method. Create the function by passing a configuration structure to a class method.

In [ ]:
config = {'name': 'myprop', 'value': -0.5, 'gain': 21}
prop = Proportional.from_config(config)
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop', 'value': -0.5, 'gain': 21}


# Viewing Functions

View the details of the function with the "summary", which prints the name, type, parameters, value and links (if any).

In [ ]:
prop.summary()

myprop Proportional | gain 21 | -0.5 


As already seen the function details can be seen by retrieving the configuration.

In [ ]:
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop', 'value': -0.5, 'gain': 21}


In [ ]:
%nbdev_export
class Variable(BaseFunction):
    "A function that returns a variable value."
    "Parameter: The variable value."
    "Links: None"
    def __init__(self, variable, value=0, name="variable"):
        super().__init__(name, value)
        self.value = variable
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")
        
    def get_config(self):
        config = super().get_config()
        return config



In [ ]:
%nbdev_export
class Constant(BaseFunction):
    "A function that returns a constant value."
    "Parameter: The constant value."
    "Links: None"
    def __init__(self, constant, value=0, name="constant"):
        super().__init__(name, value)
        self.value = constant
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")

    def get_config(self):
        config = super().get_config()


In [ ]:
%nbdev_export
class Subtract(BaseFunction):
    "A function that subtracts one value from another."
    "Parameter: None."
    "Links: Two links required to each the values to be subtracted."

    def __init__(self, name="subtract"):
        super().__init__(name)
    
    def __call__(self, verbose=False):
        #print("Sub ", self.links[0].get_value(),self.links[1].get_value() )
        self.value = self.links[0].get_value()-self.links[1].get_value()

        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        return super().get_config()
                        

In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "Integration function."
    "Parameter: The gain and slow values."
    "Links: One."

    def __init__(self, gain, slow, value=0, name="integration"):
        super().__init__(name, value)
        self.gain = gain
        self.slow = slow
    
    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = self.value +  ((input * self.gain) - self.value)/self.slow
        
        return super().__call__(verbose)

    def summary(self):
        super().summary(f'gain {self.gain} slow {self.slow} ')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        config["slow"] = self.slow
        return config       
        

# Setting Links
The next cell shows how a link is added from one function to another. In this case from an Integration function to a constant function. So, whenever "integrator" runs it will get its input from "cons". 

In [ ]:
integrator = Integration(3, 10)
cons = Constant(5)
integrator.add_link(cons)

# Running a Fucntion
A function can simply be run by calling it, without any parameters. It will use whatever input was set by the links. It returns the result of the function. In this example it will be 5 * 3 / 10, that is, input * gain / slow.

In [ ]:
output = integrator()
print(output)

1.5


# Array Input
The input to a function can be an array rather than a single value.

In [ ]:

integrator.set_value(np.array([1, 2, 4, 3]))
output = integrator()
print(output)
np.testing.assert_array_equal(output, [2.4, 3.3, 5.1, 4.2])

[2.4 3.3 5.1 4.2]


In [ ]:
print(integrator.get_config())

{'type': 'Integration', 'name': 'integration', 'value': array([2.4, 3.3, 5.1, 4.2]), 'gain': 3, 'slow': 10}


In [ ]:
integ = Integration(**{'name': 'myinteg', 'value': 1, 'gain': 20, 'slow': 100})
print(integ.get_config())
assert integ.get_config() == {'type': 'Integration', 'name': 'myinteg', 'value': 1, 'gain': 20, 'slow': 100}

{'type': 'Integration', 'name': 'myinteg', 'value': 1, 'gain': 20, 'slow': 100}


In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.
